In [ ]:
#1、查询单只股票趋势图，包括昨日和今日增减。
#2、每日跟踪持续5日，15日，30日加仓的标的。
#3、较前日增减前10名。

In [1]:
from pymongo import MongoClient
import pandas as pd
from collections import defaultdict
pd.options.display.max_columns = None
pd.options.display.max_rows = None
collection=MongoClient()['wencai']['north']
resource=collection.find()
num=int(input('输入查询的天数：'))
df=pd.DataFrame(resource).tail(num)

def get_code():
    code_list=[]
    for i in df.tail(1).iterrows():
        for j in i[1]['data']:
            code_list.append((j['股票代码'],j['股票简称']))
    return code_list

def get_net():
    all_data=[]
    for i in get_code():
        net_list=[]
        for j in df.iterrows():
            for m in j[1]['data']:
                if m['股票代码']==i[0]:
                    if m['陆股通净买入额']!=None:
                        net_list.append(m['陆股通净买入额'])
        if (len(net_list)>0) and (sum(net_list)!=0):
            all_data.append({'股票代码':i[0],'股票简称':i[1],'净买入':net_list})
    return all_data

def get_all_in_result():
    net_name=[]
    for i in get_net():
        if len([x for x in i['净买入'] if x<0])==0:
            net_name.append(i['股票简称'])
    print('在{}日内持续净流入：'.format(num))
    print(net_name)
#         elif len([x for x in i['净买入'] if x<0])==1:
#             net_name.append(i['股票简称'])
#     print('在{}日内有一日流出：'.format(num))
#     print(net_name)
#         else:
#             continue
def get_one_day_out_result():
    net_name=[]
    for i in get_net():
        if len([x for x in i['净买入'] if x<0])==1:
            net_name.append(i['股票简称'])
    print('在{}日内只有一天流出：'.format(num))
    print(net_name)
    
def get_all_out_result():
    net_name=[]
    for i in get_net():
        if len([x for x in i['净买入'] if x>0])==0:
            net_name.append(i['股票简称'])
    print('在{}日内持续净流出：'.format(num))
    print(net_name)
    
def get_one_day_in_result():
    net_name=[]
    for i in get_net():
        if len([x for x in i['净买入'] if x>0])==1:
            net_name.append(i['股票简称'])
    print('在{}日内只有一天流入：'.format(num))
    print(net_name)
    

def run():
    choice=int(input('输入选择：（持续净流入1、一日流出2、持续流出3、只有1日流入4）'))
    if choice==1:
        get_all_in_result()
    if choice==2:
        get_one_day_out_result()
    if choice==3:
        get_all_out_result()
    if choice==4:
        get_one_day_in_result()
run()

MongoClient().close()

输入查询的天数： 15
输入选择：（持续净流入1、一日流出2、持续流出3、只有1日流入4） 1


在15日内持续净流入：
['汤臣倍健', '珀莱雅', '深深房A']


In [2]:
### 每日跟踪持股比例减少/增加的前10名

from pymongo import MongoClient
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

collection=MongoClient()['wencai']['north']
resource=collection.find()
df=pd.DataFrame(resource).tail(2)
today_df=df.iloc[1]
yesterday_df=df.iloc[0]
#### 1是今天，0是昨天
ratio_change_list=[]
for i in today_df['data']:
    for j in yesterday_df['data']:
        if j['股票简称']==i['股票简称']:
            today_ratio=i['陆股通持股占流通a股比'] if i['陆股通持股占流通a股比'] is not None else 0
            yesterday_ratio=j['陆股通持股占流通a股比'] if j['陆股通持股占流通a股比'] is not None else 0
            ratio_change=round(float(today_ratio-yesterday_ratio),8)
            ratio_change_list.append((ratio_change,j['股票简称']))
bili=sorted(ratio_change_list)
print('-------------减少的前10名--------------')
print(bili[:10])
print('-------------增加的前10名--------------')
print(bili[-10:])

-------------减少的前10名--------------
[(-1.2828, '蓝盾股份'), (-1.1491, '凯利泰'), (-0.9885, '飞利信'), (-0.9258, '创世纪'), (-0.8364, '雅化集团'), (-0.803, '德利股份'), (-0.7802, '酒鬼酒'), (-0.7389, '英科医疗'), (-0.7364, '英唐智控'), (-0.7318, '药石科技')]
-------------增加的前10名--------------
[(0.4479, '凯莱英'), (0.4772, '中国外运'), (0.5175, '广发证券'), (0.5607, '国联证券'), (0.57, '首旅酒店'), (0.6534, '鲁西化工'), (0.7096, '白云机场'), (0.7998, '上海机场'), (1.0962, '云铝股份'), (2.4785, '杰瑞股份')]


In [5]:
### 查某只股票的北向资金流动趋势

from pymongo import MongoClient
import pandas as pd
import json
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.globals import CurrentConfig,NotebookType
CurrentConfig.NOTEBOOK_TYPE=NotebookType.JUPYTER_LAB

collection=MongoClient()['wencai']['north']
# stock_name='方正证券'
stock_name=input('输入名称：')

resource=collection.find()
df=pd.DataFrame(resource)
ratio_list=[]
stock_list=[]
date=[]
for i in df.iterrows():
    date.append(i[1]['date'])
    for j in i[1]['data']:
        if j['股票简称']==stock_name:
#             stock_list.append(j['陆股通持股量']/10000)
            ratio_list.append(j['陆股通持股占流通a股比'])
            
line=(
    Line()
    .add_xaxis(xaxis_data=date)
#     .add_yaxis(
#         series_name='持股数量',
#         y_axis=stock_list,
#         label_opts=opts.LabelOpts(is_show=False)
#     )
    .add_yaxis(
        series_name='持股比例',
        y_axis=ratio_list,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        toolbox_opts=opts.ToolboxOpts(),
        datazoom_opts=opts.DataZoomOpts(),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
        title_opts=opts.TitleOpts(title='{}北向资金趋势图'.format(stock_name)))
     )
line.load_javascript()
line.render_notebook()



C:\Users\yanzhan\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


输入名称： 云铝股份


C:\Users\yanzhan\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
